In [1]:
from tensorflow.keras.utils import get_file

try:
    path = get_file('GoogleNews-vectors-negative300.bin.gz', origin='https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz')
except:
    print('Error downloading')
    raise
    
print(path)

/root/.keras/datasets/GoogleNews-vectors-negative300.bin.gz


In [2]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)

In [4]:
w = model['cat']

In [5]:
w

array([ 0.0123291 ,  0.20410156, -0.28515625,  0.21679688,  0.11816406,
        0.08300781,  0.04980469, -0.00952148,  0.22070312, -0.12597656,
        0.08056641, -0.5859375 , -0.00445557, -0.296875  , -0.01312256,
       -0.08349609,  0.05053711,  0.15136719, -0.44921875, -0.0135498 ,
        0.21484375, -0.14746094,  0.22460938, -0.125     , -0.09716797,
        0.24902344, -0.2890625 ,  0.36523438,  0.41210938, -0.0859375 ,
       -0.07861328, -0.19726562, -0.09082031, -0.14160156, -0.10253906,
        0.13085938, -0.00346375,  0.07226562,  0.04418945,  0.34570312,
        0.07470703, -0.11230469,  0.06738281,  0.11230469,  0.01977539,
       -0.12353516,  0.20996094, -0.07226562, -0.02783203,  0.05541992,
       -0.33398438,  0.08544922,  0.34375   ,  0.13964844,  0.04931641,
       -0.13476562,  0.16308594, -0.37304688,  0.39648438,  0.10693359,
        0.22167969,  0.21289062, -0.08984375,  0.20703125,  0.08935547,
       -0.08251953,  0.05957031,  0.10205078, -0.19238281, -0.09

In [6]:
import numpy as np;
w1 = model['cat']
w2 = model['dog']
print(np.linalg.norm(w1-w2))

w1 = model['cat']
w2 = model['car']
print(np.linalg.norm(w1-w2))

2.0815337
3.5739133


In [7]:
model.most_similar('sachin')

[('tendulkar', 0.7856917381286621),
 ('ganguly', 0.7785738706588745),
 ('sehwag', 0.7773318886756897),
 ('dravid', 0.7691141366958618),
 ('sachin_tendulkar', 0.741614580154419),
 ('laxman', 0.7271499037742615),
 ('KrisSrikkanth_says', 0.6995106935501099),
 ('murali', 0.6993196606636047),
 ('afridi', 0.689164400100708),
 ('odi', 0.6883801817893982)]

In [8]:
model.most_similar(positive = ['king', 'woman'], negative = ['man'])

[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321243286133),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.518113374710083),
 ('sultan', 0.5098593235015869),
 ('monarchy', 0.5087411999702454)]

In [9]:
model.doesnt_match('girl wife daughter sea'.split())

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'sea'

In [10]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io
import requests
import re

In [11]:
import requests 

r = requests.get("https://ota.bodleian.ox.ac.uk/repository/xmlui/bitstream/handle/20.500.12024/5730/5730.txt")
raw_text = r.text.lower()

print(raw_text[0:10000])

treasure island
to s.l.o., an american gentleman in accordance with whose classic taste the following narrative has been designed, it is now, in return for numerous delightful hours, and with the kindest wishes, dedicated by his affectionate friend, the author.
to the hesitating purchaser
if sailor tales to sailor tunes,
  storm and adventure, heat and cold,
if schooners, islands, and maroons,
  and buccaneers, and buried gold,
and all the old romance, retold
  exactly in the ancient way,
can please, as me they pleased of old,
  the wiser youngsters of today:
part one—the old buccaneer
the old sea-dog at the admiral benbow
squire trelawney, dr. livesey, and the rest of these gentlemen having asked me to write down the whole particulars about treasure island, from the beginning to the end, keeping nothing back but the bearings of the island, and that only because there is still treasure not yet lifted, i take up my pen in the year of grace 17__ and go back to the time when my father kep

In [13]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(raw_text)
vocab = set()
tokenized_text = []

for token in doc:
    word = ''.join([i if ord(i) < 128 else ' ' for i in token.text])
    word = word.strip()
    if not token.is_digit \
        and not token.like_url \
        and not token.like_email:
        vocab.add(word)
        tokenized_text.append(word)
        
print(f"Vocab size: {len(vocab)}")

Vocab size: 5904


In [14]:
tokenized_text

['treasure',
 'island',
 '',
 'to',
 's.l.o',
 '.',
 ',',
 'an',
 'american',
 'gentleman',
 'in',
 'accordance',
 'with',
 'whose',
 'classic',
 'taste',
 'the',
 'following',
 'narrative',
 'has',
 'been',
 'designed',
 ',',
 'it',
 'is',
 'now',
 ',',
 'in',
 'return',
 'for',
 'numerous',
 'delightful',
 'hours',
 ',',
 'and',
 'with',
 'the',
 'kindest',
 'wishes',
 ',',
 'dedicated',
 'by',
 'his',
 'affectionate',
 'friend',
 ',',
 'the',
 'author',
 '.',
 '',
 'to',
 'the',
 'hesitating',
 'purchaser',
 '',
 'if',
 'sailor',
 'tales',
 'to',
 'sailor',
 'tunes',
 ',',
 '',
 'storm',
 'and',
 'adventure',
 ',',
 'heat',
 'and',
 'cold',
 ',',
 '',
 'if',
 'schooners',
 ',',
 'islands',
 ',',
 'and',
 'maroons',
 ',',
 '',
 'and',
 'buccaneers',
 ',',
 'and',
 'buried',
 'gold',
 ',',
 '',
 'and',
 'all',
 'the',
 'old',
 'romance',
 ',',
 'retold',
 '',
 'exactly',
 'in',
 'the',
 'ancient',
 'way',
 ',',
 '',
 'can',
 'please',
 ',',
 'as',
 'me',
 'they',
 'pleased',
 'of',
 '

In [15]:
print(list(vocab)[:20])

['', 'glisten', 'violence', 'bustling', 'carts', 'surprised', 'feeling', 'babby', 'thrusting', 'naked', 'pillow', 'attracted', 'refused', 'grinding', 'loophole', 'initial', 'woke', 'ones', 'bird', 'forefoot']


In [16]:
dict(enumerate(vocab))

{0: '',
 1: 'glisten',
 2: 'violence',
 3: 'bustling',
 4: 'carts',
 5: 'surprised',
 6: 'feeling',
 7: 'babby',
 8: 'thrusting',
 9: 'naked',
 10: 'pillow',
 11: 'attracted',
 12: 'refused',
 13: 'grinding',
 14: 'loophole',
 15: 'initial',
 16: 'woke',
 17: 'ones',
 18: 'bird',
 19: 'forefoot',
 20: 'third',
 21: 'flint',
 22: 'calling',
 23: 'slights',
 24: 'clouds',
 25: 'row',
 26: 'suspicions',
 27: 'submit',
 28: 'plan',
 29: 'cheerful',
 30: 'sails',
 31: 'seas',
 32: 'continuously',
 33: 'push',
 34: 'whisper "i\'m',
 35: ',',
 36: 'slip',
 37: 'housekeeper',
 38: 'equals',
 39: 'warded',
 40: 'beats',
 41: 'starved',
 42: 'hostage',
 43: 'gently',
 44: 'meddling',
 45: 'and',
 46: 'hilt',
 47: 'capstan',
 48: 'cutwater',
 49: 'stones',
 50: 'reader',
 51: 'perked',
 52: 'uppermost',
 53: 'flight',
 54: 'aroused',
 55: 'sir" laying',
 56: 'cocks',
 57: 'pen',
 58: 'mountings',
 59: 'companion',
 60: 'frayed',
 61: 'flowery',
 62: 'smell',
 63: 'priming',
 64: 'hot',
 65: 'avas

In [17]:
word2idx = dict((n, v) for v, n in enumerate(vocab))
idx2word = dict((n, v) for n, v in enumerate(vocab))

In [18]:
word2idx

{'': 0,
 'glisten': 1,
 'violence': 2,
 'bustling': 3,
 'carts': 4,
 'surprised': 5,
 'feeling': 6,
 'babby': 7,
 'thrusting': 8,
 'naked': 9,
 'pillow': 10,
 'attracted': 11,
 'refused': 12,
 'grinding': 13,
 'loophole': 14,
 'initial': 15,
 'woke': 16,
 'ones': 17,
 'bird': 18,
 'forefoot': 19,
 'third': 20,
 'flint': 21,
 'calling': 22,
 'slights': 23,
 'clouds': 24,
 'row': 25,
 'suspicions': 26,
 'submit': 27,
 'plan': 28,
 'cheerful': 29,
 'sails': 30,
 'seas': 31,
 'continuously': 32,
 'push': 33,
 'whisper "i\'m': 34,
 ',': 35,
 'slip': 36,
 'housekeeper': 37,
 'equals': 38,
 'warded': 39,
 'beats': 40,
 'starved': 41,
 'hostage': 42,
 'gently': 43,
 'meddling': 44,
 'and': 45,
 'hilt': 46,
 'capstan': 47,
 'cutwater': 48,
 'stones': 49,
 'reader': 50,
 'perked': 51,
 'uppermost': 52,
 'flight': 53,
 'aroused': 54,
 'sir" laying': 55,
 'cocks': 56,
 'pen': 57,
 'mountings': 58,
 'companion': 59,
 'frayed': 60,
 'flowery': 61,
 'smell': 62,
 'priming': 63,
 'hot': 64,
 'avast': 

In [19]:
idx2word

{0: '',
 1: 'glisten',
 2: 'violence',
 3: 'bustling',
 4: 'carts',
 5: 'surprised',
 6: 'feeling',
 7: 'babby',
 8: 'thrusting',
 9: 'naked',
 10: 'pillow',
 11: 'attracted',
 12: 'refused',
 13: 'grinding',
 14: 'loophole',
 15: 'initial',
 16: 'woke',
 17: 'ones',
 18: 'bird',
 19: 'forefoot',
 20: 'third',
 21: 'flint',
 22: 'calling',
 23: 'slights',
 24: 'clouds',
 25: 'row',
 26: 'suspicions',
 27: 'submit',
 28: 'plan',
 29: 'cheerful',
 30: 'sails',
 31: 'seas',
 32: 'continuously',
 33: 'push',
 34: 'whisper "i\'m',
 35: ',',
 36: 'slip',
 37: 'housekeeper',
 38: 'equals',
 39: 'warded',
 40: 'beats',
 41: 'starved',
 42: 'hostage',
 43: 'gently',
 44: 'meddling',
 45: 'and',
 46: 'hilt',
 47: 'capstan',
 48: 'cutwater',
 49: 'stones',
 50: 'reader',
 51: 'perked',
 52: 'uppermost',
 53: 'flight',
 54: 'aroused',
 55: 'sir" laying',
 56: 'cocks',
 57: 'pen',
 58: 'mountings',
 59: 'companion',
 60: 'frayed',
 61: 'flowery',
 62: 'smell',
 63: 'priming',
 64: 'hot',
 65: 'avas

In [20]:
tokenized_text = [word2idx[word] for word in tokenized_text]

In [21]:
tokenized_text

[5544,
 2208,
 0,
 4669,
 4042,
 4730,
 35,
 2738,
 4854,
 2462,
 4208,
 4924,
 3275,
 281,
 5113,
 4679,
 1382,
 3830,
 5638,
 3981,
 2004,
 5740,
 35,
 1361,
 4101,
 1541,
 35,
 4208,
 5521,
 1155,
 2688,
 72,
 3176,
 35,
 45,
 3275,
 1382,
 1091,
 4120,
 35,
 692,
 1918,
 5342,
 1459,
 4092,
 35,
 1382,
 855,
 4730,
 0,
 4669,
 1382,
 3951,
 5824,
 0,
 3477,
 1292,
 4956,
 4669,
 1292,
 5284,
 35,
 0,
 3839,
 45,
 585,
 35,
 598,
 45,
 5718,
 35,
 0,
 3477,
 295,
 35,
 5170,
 35,
 45,
 1404,
 35,
 0,
 45,
 1471,
 35,
 45,
 639,
 5901,
 35,
 0,
 45,
 3028,
 1382,
 3053,
 1632,
 35,
 3842,
 0,
 1071,
 4208,
 1382,
 5238,
 3141,
 35,
 0,
 5218,
 876,
 35,
 1313,
 4499,
 2724,
 1182,
 3177,
 3053,
 35,
 0,
 1382,
 3188,
 3985,
 3177,
 4711,
 3549,
 0,
 4931,
 1266,
 0,
 1382,
 3053,
 4979,
 0,
 1382,
 3053,
 3002,
 4420,
 5404,
 3528,
 1382,
 851,
 2483,
 0,
 2331,
 3701,
 35,
 3613,
 4730,
 5585,
 35,
 45,
 1382,
 316,
 3177,
 5322,
 3121,
 4517,
 5166,
 4499,
 4669,
 2800,
 5297,
 138

In [22]:
# cut the text in semi-redundant sequences of maxlen words
maxlen = 6
step = 3
sentences = []
next_words = []
for i in range(0, len(tokenized_text) - maxlen, step):
    sentences.append(tokenized_text[i: i + maxlen])
    next_words.append(tokenized_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 28500


In [23]:
sentences[0:5]

[[5544, 2208, 0, 4669, 4042, 4730],
 [4669, 4042, 4730, 35, 2738, 4854],
 [35, 2738, 4854, 2462, 4208, 4924],
 [2462, 4208, 4924, 3275, 281, 5113],
 [3275, 281, 5113, 4679, 1382, 3830]]

In [24]:
next_words[0:5]

[35, 2462, 3275, 4679, 5638]

In [25]:
import numpy as np

print('One hot Vectorization...')
x = np.zeros((len(sentences), maxlen, len(vocab)), dtype=np.bool)
y = np.zeros((len(sentences), len(vocab)), dtype=np.bool)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence):
    x[i, t, word] = 1
  y[i, next_words[i]] = 1

One hot Vectorization...


In [26]:
x.shape

(28500, 6, 5904)

In [27]:
y.shape

(28500, 5904)

In [28]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(vocab)), return_sequences = True))
model.add(LSTM(16, input_shape=(maxlen, len(vocab)), return_sequences = False))
model.add(Dense(len(vocab), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 6, 128)            3088896   
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                9280      
_________________________________________________________________
dense (Dense)                (None, 5904)              100368    
Total params: 3,198,544
Trainable params: 3,198,544
Non-trainable params: 0
_________________________________________________________________


In [31]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("******************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(tokenized_text) - maxlen)
    #generated = ''
    sentence = tokenized_text[start_index: start_index + maxlen]
    #generated += sentence
    o = ' '.join([idx2word[idx] for idx in sentence])
    print(f'----- Generating with seed: "{o}"')
    #sys.stdout.write(generated)

    for i in range(100):
        x_pred = np.zeros((1, maxlen, len(vocab)))
        for t, word in enumerate(sentence):
            x_pred[0, t, word] = 1.
            

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = np.argmax(preds)
        next_word = idx2word[next_index]

        #generated += next_char
        sentence = sentence[1:]
        sentence.append(next_index)

        sys.stdout.write(next_word)
        sys.stdout.write(' ') 
        sys.stdout.flush()
    print()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
model.fit(x, y, batch_size=128, epochs=60, callbacks=[print_callback])

Epoch 1/60
223/223 [==============================] - 35s 11ms/step - loss: 6.5246
******************************************************
----- Generating text after Epoch: 0
----- Generating with seed: "really believe his presence did us"
, the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , the , , 
Epoch 2/60
223/223 [==============================] - 2s 11ms/step - loss: 5.7166
******************************************************
----- Generating text after Epoch: 1
----- Generating with seed: "end into a great library ,"
 " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " " 
Epoch 3/60
223/223 [==============================] - 2s 11ms/